<a href="https://colab.research.google.com/github/minhbao1705/pytorch_tutorial/blob/main/template.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
import torch
import torchvision

import torch.nn as nn
import torchvision.transforms as transforms
import numpy as np

# Ví dụ về Autograd

In [3]:
# Xây dựng đồ thị tính toán với phương trình: y = 5*x + 10
x = torch.tensor(5., requires_grad=True)
w = torch.tensor(3., requires_grad=True)
b = torch.tensor(10., requires_grad=True)

y = w*x + b

# Thực hiện tình gradients
y.backward()

print("x.grad: ", x.grad)
print("w.grad: ", w.grad)
print("b.grad: ", b.grad)

x.grad:  tensor(3.)
w.grad:  tensor(5.)
b.grad:  tensor(1.)


# Ví dụ về Autograd (tiếp theo)

In [4]:
# Tạo 2 tensor x, y có kích thước lần lượt là (10, 4) và (10, 3)
x = torch.randn(10, 4)
y = torch.randn(10, 3)

# Tạo fully connected layer
linear_layer = nn.Linear(4, 3)
print("w: ", linear_layer.weight)
print("b: ", linear_layer.bias)

# Xây dựng hàm loss và optimizer
criterion = nn.MSELoss()
optimizer = torch.optim.SGD(linear_layer.parameters(), lr=0.001)

# Lan truyền tiến 
forward = linear_layer(x)

# Tính giá trị loss
loss = criterion(forward, y)
print("loss: ", loss.item())

# Lan truyền ngược
loss.backward()

# Xuất giá trị đạo hàm
print("dL/dW: ", linear_layer.weight.grad)
print("dL/dB: ", linear_layer.bias.grad)

optimizer.step()

# Xuất kết quả sau một bước đạo hàm
pred = linear_layer(x)
loss = criterion(pred, y)
print('loss after 1 step optimization: ', loss.item())

w:  Parameter containing:
tensor([[ 0.1417,  0.1743, -0.2942, -0.4610],
        [ 0.4749, -0.3777, -0.3547, -0.3145],
        [-0.0052,  0.2074,  0.4225, -0.0094]], requires_grad=True)
b:  Parameter containing:
tensor([-0.3900, -0.4351,  0.4063], requires_grad=True)
loss:  0.9158770442008972
dL/dW:  tensor([[-0.0846, -0.0261,  0.1516, -0.0202],
        [-0.1547, -0.4298, -0.1864, -0.0719],
        [ 0.0193,  0.0355,  0.0591, -0.0121]])
dL/dB:  tensor([-0.3548, -0.0386,  0.1787])
loss after 1 step optimization:  0.9154328107833862


# Load dữ liệu với numpy

In [5]:
x = np.array([[1, 2], [3, 4]])

# Chuyển đổi numpy array sang torch tensor
y = torch.from_numpy(x)
print("type(y): ", type(y))

# Chuyển đổi torch tensor sang numpy array
z = y.numpy()
print("type(z): ", type(z))

type(y):  <class 'torch.Tensor'>
type(z):  <class 'numpy.ndarray'>


# Input pipeline

In [8]:
# Sử dụng dữ liệu với torchvision.datasets
train_dataset = torchvision.datasets.CIFAR10(root="./data",
                                             train=True,
                                             transform=transforms.ToTensor(),
                                             download=True)

image, label = train_dataset[0]
print("size: ", image.size())
print("label: ", label)

# DataLoader
train_loader = torch.utils.data.DataLoader(dataset=train_dataset,
                                           batch_size=64,
                                           shuffle=True)
# Khi bắt đầu lặp, sẽ lần lượt đọc dữ liệu đưa vào queue và thread
data_iter = iter(train_loader)

# Thực hiện mini-batch trên các ảnh và nhãn tương ứng
images, labels = next(data_iter)

# Quá trình load ảnh thực tế được thực hiện nhưu sau
for image, label in train_loader:
  # Quá trình huấn luyện được thực hiện tại đây
  pass



Files already downloaded and verified
size:  torch.Size([3, 32, 32])
label:  6


# Input pipeline for custom dataset

In [9]:
# Xây dựng hàm xử lý cho custom dataset
class CustomDataset(object): # torch.utils.data.Dataset
  def __init__(self):
    # Khởi tạo danh sách đường dẫn hình hoặc tên hình
    pass

  def get_item(self, index):
    # Đọc data từ file (numpy.fromfile, PIL.Image.open)
    # Tiền xử lí dữ liệu (torchvision.Transform)
    # Trả về một cặp dữ liệu (image và label)
    pass
  
  def __len__(self):
    # Trả về số lượng mẫu trong dữ liệu
    pass

# Pretrained model

In [12]:
# Sử dụng pretrained ResNet-18
resnet = torchvision.models.resnet18(pretrained=True)

# Finetune model
for param in resnet.parameters():
  param.requires_grad = False

# Thay thế top ko layer đầu tiên cho finetuning
resnet.fc = nn.Linear(resnet.fc.in_features, 100) # k = 100

# Lan truyền tiến
image = torch.rand(64, 3, 224, 224)
outputs = resnet(image)
print("output size: ", outputs.size())

/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet18_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet18_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


output size:  torch.Size([64, 100])


# Save and Load model

In [13]:
torch.save(resnet, "model_resnet18.ckpt")
model = torch.load('model_resnet18.ckpt')
print("load model....")

# Save and load only the model parameters (thường dùng).
torch.save(resnet.state_dict(), 'params_resnet.ckpt')
resnet.load_state_dict(torch.load('params_resnet.ckpt'))
print("load params.....")

load model....
load params.....
